In [1]:
%load_ext autoreload
%autoreload 2
from utils.models import *
import utils.api as api
CACHED_HOURLY_DATA = 'cached_hourly_data.json'
CACHED_FORCAST_DATA = 'cached_forecast_data.json'

In [26]:
api.cache_forecast(api.fetch_hourly_forecast(), CACHED_HOURLY_DATA)

Forecast data saved to forecast.json


In [4]:
data = api.load_cached_data(CACHED_HOURLY_DATA)

In [10]:
m = GridpointHourlyForecastGeoJson.model_validate(data)
m.properties.periods[0]

GridpointHourlyForecastPeriod(number=1, name='This Afternoon', startTime='2025-06-20T14:00:00-04:00', endTime='2025-06-20T18:00:00-04:00', isDaytime=True, temperature=80, temperatureUnit='F', temperatureTrend='', probabilityOfPrecipitation=QuantitativeValue(value=10, unitCode='wmoUnit:percent'), windSpeed='2 to 6 mph', windDirection='NNW', icon='https://api.weather.gov/icons/land/day/bkn?size=medium', shortForecast='Partly Sunny', detailedForecast='Partly sunny, with a high near 80. North northwest wind 2 to 6 mph.')